In [31]:
import os
import numpy as np
from pathlib import Path

import pandas as pd
import sys
# kouchou-aiのパスを追加
sys.path.append("../kouchou-ai/server")
sys.path.append("../kouchou-ai/server/broadlistening/pipeline/")

In [32]:
slug = "aipubcom"
exp_prefix = f"20250415_プロンプト修正_{slug}"
DATA_DIR = Path("../data/original_outputs")# 広聴AIで出力したデータを保存しているディレクトリ
OUTPUT_DIR = Path("../data/experiment_outputs")

# クラスタリング

In [33]:
from umap import UMAP
from broadlistening.pipeline.steps.hierarchical_clustering import hierarchical_clustering_embeddings

arguments_df = pd.read_csv(DATA_DIR / slug /  "args.csv")
embeddings_df = pd.read_pickle(DATA_DIR / slug / "embeddings.pkl")
embeddings_array = np.asarray(embeddings_df["embedding"].values.tolist())
cluster_nums = [15, 70]
n_samples = embeddings_array.shape[0]
default_n_neighbors = 15

if n_samples <= default_n_neighbors:
    n_neighbors = max(2, n_samples - 1)
else:
    n_neighbors = default_n_neighbors

umap_model = UMAP(random_state=42, n_components=2, n_neighbors=n_neighbors)
umap_embeds = umap_model.fit_transform(embeddings_array)


/Users/nasuka/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/nasuka/.pyenv/versions/3.12.4/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [34]:
# 広聴AIで出力したレポートのslug
cluster_results = hierarchical_clustering_embeddings(
    umap_embeds=umap_embeds,
    cluster_nums=cluster_nums,
)
result_df = pd.DataFrame(
    {
        "arg-id": arguments_df["arg-id"],
        "argument": arguments_df["argument"],
        "x": umap_embeds[:, 0],
        "y": umap_embeds[:, 1],
    }
)
for cluster_level, final_labels in enumerate(cluster_results.values(), start=1):
    result_df[f"cluster-level-{cluster_level}-id"] = [f"{cluster_level}_{label}" for label in final_labels]



start initial clustering
end initial clustering
start hierarchical clustering
[15, 70]
n_cluster_cut:  15
end hierarchical clustering


In [35]:
clusters_argument_df =  result_df.copy()
orig_initial_label_df = pd.read_csv(DATA_DIR / slug / "hierarchical_initial_labels.csv")
orig_merge_label_df = pd.read_csv(DATA_DIR / slug / "hierarchical_merge_labels.csv")

exp_output_dir = OUTPUT_DIR / exp_prefix

os.makedirs(exp_output_dir, exist_ok=True)


# プロンプト修正の実験

## 初期ラベリング

In [36]:
INITIAL_PROMPT = """あなたはKJ法が得意なデータ分析者です。userのinputはグループに集まったラベルです。なぜそのラベルが一つのグループであるか解説し、表札（label）をつけてください。
表札については、グループ内の具体的な論点や特徴を反映した、具体性の高い名称を考案してください。  
出力はJSONとし、フォーマットは以下のサンプルを参考にしてください。


# サンプルの入出力
## 入力例
- 手作業での意見分析は時間がかかりすぎる。AIで効率化できると嬉しい
- 今のやり方だと分析に工数がかかりすぎるけど、AIならコストをかけずに分析できそう
- AIが自動で意見を整理してくれると楽になって嬉しい


## 出力例
{{
    "label": "AIによる業務効率の大幅向上とコスト効率化",
    "description": "このクラスタは、従来の手作業による意見分析と比較して、AIによる自動化で分析プロセスが効率化され、作業時間の短縮や運用コストの効率化が実現される点に対する前向きな評価が中心です。"
}}"""

In [37]:
from broadlistening.pipeline.steps.hierarchical_initial_labelling import initial_labelling

In [38]:
cluster_id_columns = [col for col in clusters_argument_df.columns if col.startswith("cluster-level-")]
initial_cluster_id_column = cluster_id_columns[-1]
sampling_num = 30
initial_labelling_prompt = INITIAL_PROMPT
model = "gpt-4o"
workers = 70

initial_label_df = initial_labelling(
    initial_labelling_prompt,
    clusters_argument_df,
    sampling_num,
    model,
    workers,
)

In [39]:
# 最下層のクラスタ。プロンプト修正後
initial_label_df.head()

,cluster_id,label,description
0,2_34,AI生成技術と著作権・倫理問題に関する懸念と規制の必要性,このクラスタは、AIによる画像生成技術がクリエイターの著作権を侵害し、無断で作品が使用される...
1,2_56,生成AIによるクリエイターへの影響と規制の必要性,このクラスタは、生成AIがイラストやデザインを生業とするクリエイターに与える影響についての懸...
2,2_28,生成AIによるクリエイターへの影響と懸念,このクラスタは、生成AIの普及がクリエイターに与える多面的な影響に対する懸念を中心にしていま...
3,2_61,生成AIと日本文化の保護に関する懸念と規制の必要性,このクラスタは、生成AI技術が日本の文化、特にアニメやマンガといったサブカルチャーに与える影...
4,2_14,AI生成物と手描き作品の識別と著作権保護の必要性,このクラスタは、AIによるイラスト生成が進化する中で、手描き作品とAI生成物の区別が難しくな...


In [40]:
initial_clusters_argument_df = clusters_argument_df.merge(
    initial_label_df,
    left_on=initial_cluster_id_column,
    right_on="cluster_id",
    how="left",
).rename(
    columns={
        "label": f"{initial_cluster_id_column.replace('-id', '')}-label",
        "description": f"{initial_cluster_id_column.replace('-id', '')}-description",
    }
)

In [41]:
initial_clusters_argument_df

,arg-id,argument,x,y,cluster-level-1-id,cluster-level-2-id,cluster_id,cluster-level-2-label,cluster-level-2-description
0,A59d26584-95a8-481b-826e-9a86d30e31d4_0,特定のクリエイターの画風がAIによって複製され、悪意のある創作に利用される可能性があるため、...,0.375076,9.062553,1_8,2_34,2_34,AI生成技術と著作権・倫理問題に関する懸念と規制の必要性,このクラスタは、AIによる画像生成技術がクリエイターの著作権を侵害し、無断で作品が使用される...
1,A59d26584-95a8-481b-826e-9a86d30e31d4_1,生成AIによって画風が再現・量産されることで、クリエイターの画風や画力の希少価値が下がり、顧...,1.399587,8.022738,1_12,2_56,2_56,生成AIによるクリエイターへの影響と規制の必要性,このクラスタは、生成AIがイラストやデザインを生業とするクリエイターに与える影響についての懸...
2,A59d26584-95a8-481b-826e-9a86d30e31d4_2,生成AIの登場により、活動の意義や需要を失ったクリエイターが活動を断念するケースが増えており...,0.902624,10.476227,1_9,2_28,2_28,生成AIによるクリエイターへの影響と懸念,このクラスタは、生成AIの普及がクリエイターに与える多面的な影響に対する懸念を中心にしていま...
3,A59d26584-95a8-481b-826e-9a86d30e31d4_3,日本のクリエイターの活動と創作活動の発展のために、画風の保護と生成AIの使用に関する規制が必...,1.696006,11.296448,1_7,2_61,2_61,生成AIと日本文化の保護に関する懸念と規制の必要性,このクラスタは、生成AI技術が日本の文化、特にアニメやマンガといったサブカルチャーに与える影...
4,A4c34e2ee-1e19-47b4-88b1-4d5232fda9f1_0,好きな作家が生成AIに模倣されて創作意欲を失うのは悲しいので、生成AIは規制してほしい,2.454115,4.803548,1_13,2_14,2_14,AI生成物と手描き作品の識別と著作権保護の必要性,このクラスタは、AIによるイラスト生成が進化する中で、手描き作品とAI生成物の区別が難しくな...
...,...,...,...,...,...,...,...,...,...
25225,Ab8495764-38fb-4935-ade0-544c00bc27b6_0,他者の作品を学習素材とした生成AIの使用は違法とすべき,-1.027124,8.351385,1_8,2_55,2_55,生成AIによる著作権侵害とクリエイターの権利保護の必要性,このクラスタは、生成AIが既存の作品を無断で学習し、著作権を侵害することに対する懸念と、クリ...
25226,Ab8495764-38fb-4935-ade0-544c00bc27b6_1,生成AIは他者の著作物のモンタージュに過ぎず、盗作の寄せ集めである,-0.913003,10.382530,1_10,2_59,2_59,生成AIと著作権に関する倫理的・法的懸念,このクラスタは、生成AIの利用における著作権侵害や倫理的問題に対する懸念を中心にしています。...
25227,Ab8495764-38fb-4935-ade0-544c00bc27b6_2,生成AIの使用を許可すると創作業界が模造コピー品で溢れる,-0.080795,9.903406,1_9,2_2,2_2,生成AIによる著作権侵害とクリエイターの権利保護の必要性,このクラスタは、生成AIの利用がクリエイターの著作権を侵害し、創作活動に悪影響を及ぼす可能性...
25228,Ab8495764-38fb-4935-ade0-544c00bc27b6_3,生成AIを用いる場合、学習素材は製作者自身の創作物に限定すべき,-1.398726,8.439795,1_8,2_8,2_8,生成AIの著作権と倫理的利用に関する懸念と規制の必要性,このクラスタは、生成AIの学習における著作権侵害の問題に焦点を当てています。具体的には、著作...


In [42]:
initial_clusters_argument_df.to_csv(exp_output_dir / "hierarchical_initial_labels.csv", index=False)

## 統合ラベリング

In [43]:
from broadlistening.pipeline.steps.hierarchical_merge_labelling import _filter_id_columns, merge_labelling, melt_cluster_data, _build_parent_child_mapping, calculate_cluster_density

In [44]:
MERGE_PROMPT = """あなたはデータ分析のエキスパートです。
現在、テキストデータの階層クラスタリングを行っています。
下層のクラスタのタイトルと説明を与えるので、それらのクラスタが所属する上層のクラスタのタイトルと説明を作成してください。

# 指示
- 統合後のクラスタ名は、統合前のクラスタ名称をそのまま引用せず、内容に基づいた新たな名称にしてください。  
- タイトルには、具体的な事象・行動（例：地域ごとの迅速対応、復興計画の着実な進展、効果的な情報共有・地域協力など）を含めてください
  - 可能な限り具体的な表現を用いるようにし、抽象的な表現は避けてください
    - 「多様な意見」などの抽象的な表現は避けてください
- 出力例に示したJSON形式で出力してください


# サンプルの入出力
## 入力例
- 「顧客フィードバックの自動集約」: このクラスタは、SNSやオンラインレビューなどから集めた大量の意見をAIが瞬時に解析し、企業が市場のトレンドや顧客の要望を即時に把握できる点についての期待を示しています。
- 「AIによる業務効率の大幅向上とコスト効率化」: このクラスタは、従来の手作業による意見分析と比較して、AIによる自動化で分析プロセスが効率化され、作業時間の短縮や運用コストの効率化が実現される点に対する前向きな評価が中心です。

## 出力例
{
    "label": "AI技術の導入による意見分析の効率化への期待",
    "description": "大量の意見やフィードバックから迅速に洞察を抽出できるため、企業や自治体が消費者や市民の声を的確に把握し、戦略的な意思決定やサービス改善が可能になります。また、従来の手法と比べて作業負荷が軽減され、業務効率の向上やコスト削減といった実際の便益が得られると期待されています。"
}"""

In [45]:
config = {}
config["hierarchical_merge_labelling"] = {
    "prompt": MERGE_PROMPT,
    "model": "gpt-4o",
    "sampling_num": 50,
    "workers": 70,
}

In [46]:
clusters_df = initial_clusters_argument_df.copy()
cluster_id_columns: list[str] = _filter_id_columns(clusters_df.columns)
merge_result_df = merge_labelling(
    clusters_df=clusters_df,
    cluster_id_columns=sorted(cluster_id_columns, reverse=True),
    config=config,
)

100%|██████████| 1/1 [00:09<00:00,  9.02s/it]


In [47]:
melted_df = melt_cluster_data(merge_result_df)
parent_child_df = _build_parent_child_mapping(merge_result_df, cluster_id_columns)
melted_df = melted_df.merge(parent_child_df, on=["level", "id"], how="left")
melted_df.head(20)

,level,id,label,description,value,parent
0,1,1_8,生成AI技術の法的整備と倫理的利用の推進,生成AI技術の進展に伴い、著作権侵害や倫理的問題が顕在化しており、これに対する法的整備と倫理...,2557,0
1,1,1_12,生成AIによるクリエイティブ業界への影響と法的・倫理的対応の必要性,生成AIの進化に伴い、イラストレーターやクリエイターの作品が無断で学習され、模倣や盗作が行わ...,2323,0
2,1,1_9,生成AIの普及によるクリエイターの権利保護と文化維持への緊急対応の必要性,生成AIの急速な普及は、クリエイターの著作権侵害や職業的アイデンティティの喪失、創作文化の衰...,1837,0
3,1,1_7,生成AIの文化的影響と法的規制の強化に向けた取り組み,生成AI技術が日本の文化、特にアニメやマンガといったサブカルチャーに与える影響についての懸念...,2848,0
4,1,1_13,AI生成物と手描き作品の識別と著作権保護の必要性,このクラスタは、AIによるイラスト生成が進化する中で、手描き作品とAI生成物の区別が難しくな...,231,0
5,1,1_15,AI技術の進化によるクリエイターの権利侵害と倫理的課題への対応,AI技術の急速な発展により、イラストや音楽、文章などのクリエイティブな作品が無断で学習・生成...,1842,0
6,1,1_4,AI技術の進化に伴う著作権とデータ利用の法的枠組みの再構築,AIによる生成物や学習データの利用に関する著作権問題が浮上しており、クリエイターの権利保護と...,2258,0
7,1,1_2,日本の創作文化とクリエイターの権利保護に向けた法整備と支援の強化,日本の漫画、アニメ、ゲームなどの文化的コンテンツを支えるクリエイターの権利を守るため、法的保...,1618,0
8,1,1_14,AI生成技術の透明性向上と倫理的規制の強化,AI生成物の透明性を確保し、信頼性を高めるために、生成物であることの明示や表記義務の導入が求...,1951,0
9,1,1_1,著作権保護の強化と創作者の権利擁護に向けた法的・倫理的枠組みの整備,著作権侵害に対する法的保護の強化と、創作者の権利を守るための法整備が求められています。具体的...,1826,0


In [48]:
melted_df.to_csv(exp_output_dir / "プロンプト変更後のラベリング結果.csv", index=False)

In [49]:
# 自動評価

In [50]:
import json
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
random.seed(42)

from broadlistening.pipeline.services.llm import request_to_chat_openai

In [51]:
EVALUATION_PROMPT = """あなたはテキストクラスタリングの専門家です。クラスタリング結果のタイトルの適切性を評価してください。

# 指示
与えられた情報に基づいて、クラスタのタイトルがそのクラスタに含まれるテキストデータをどれだけ適切に表現しているかを評価してください。
評価結果は出力例に記載したjson形式で出力してください。

# 入力情報
1. 評価対象クラスタのタイトル
2. 評価対象クラスタに所属するテキストデータのサンプル
3. 他のクラスタからサンプリングしたテキストデータ

# 評価基準
以下の観点から、クラスタのタイトルの適切性を1〜5の5段階で評価してください：

1. inclusiveness_score: タイトルがクラスタ内のすべてのテキストの共通テーマを捉えているか（1: 全く捉えていない 〜 5: 完全に捉えている）
2. specificity_score: タイトルがこのクラスタを他のクラスタと明確に区別できるか（1: 全く区別できない 〜 5: 完全に区別できる）
3. concreteness_score: タイトルが抽象的すぎず、具体的な内容を示しているか（1: 非常に抽象的 〜 5: 非常に具体的）

# 評価例

## 入力例
評価対象クラスタのタイトル: 「災害対応の迅速化と情報共有の強化」

評価対象クラスタに所属するテキストデータのサンプル:
- 災害発生時の初動対応をもっと迅速にしてほしい。特に高齢者への支援が遅れがちだ。
- 避難所の情報がリアルタイムで更新されず、どこに行けばいいのか混乱した。
- 災害時の情報共有システムを強化し、住民が必要な情報にすぐアクセスできるようにすべき。
- 地域ごとの災害対策本部の連携が不十分で、支援物資の配布に無駄が生じていた。
- 災害発生から避難指示が出るまでのタイムラグを短縮する必要がある。

他のクラスタからサンプリングしたテキストデータ:
- 復興予算の使い道をもっと透明化してほしい。どこにいくら使われているのか不明確だ。
- 被災地域の経済復興のための長期的な計画が見えてこない。
- 防災教育を学校カリキュラムに積極的に取り入れるべきだ。
- ボランティアの受け入れ体制が整っておらず、せっかくの支援の手が活かされていない。

## 出力例
{{
  "inclusiveness_score": {{
      "score": 4,
      "reason": "タイトル「災害対応の迅速化と情報共有の強化」は、クラスタ内のテキストの主要なテーマである「初動対応の迅速化」「情報共有の問題」「対策本部の連携」などの要素を広くカバーしています。ただし、「高齢者への支援」という具体的な対象については明示されていないため、完全ではありません。"
  }},
  "specificity_score": {{
      "score": 4,
      "reason": "このタイトルは「対応の迅速化」と「情報共有」という2つの明確な焦点を持っており、他のクラスタ（予算の透明化、経済復興、防災教育など）とは明確に区別できます。ただし、災害対応の中でも「初動」に関する内容が多いため、もう少し特化した表現があるとより区別しやすくなります。"
  }},
  "concreteness_score": {{
      "score": 3,
      "reason": "「迅速化」と「情報共有の強化」という方向性は示されていますが、どのような迅速化（初動対応、避難指示など）なのか、どのような情報共有（避難所情報、支援物資など）なのかまでは具体化されていません。"
  }},
}}
"""

In [52]:
def sample_cluster_data(
    df: pd.DataFrame,
    cluster_column: str,
    cluster_id: str,
    text_column: str,
    sample_size: int
) -> list[str]:
    """クラスタからデータをサンプリングする

    Args:
        df: クラスタリング結果のDataFrame
        cluster_column: クラスタIDが格納されている列名
        cluster_id: サンプリング対象のクラスタID
        text_column: テキストデータが格納されている列名
        sample_size: サンプリングするデータ数

    Returns:
        サンプリングしたテキストデータのリスト
    """
    cluster_data = df[df[cluster_column] == cluster_id]
    sample_size = min(sample_size, len(cluster_data))
    
    if sample_size == 0:
        return []
    
    return cluster_data.sample(sample_size)[text_column].tolist()


def sample_other_clusters_data(
    df: pd.DataFrame,
    cluster_column: str,
    exclude_cluster_id: str,
    text_column: str,
    sample_size: int,
    sample_clusters: int | None = None
) -> list[str]:
    """他のクラスタからデータをサンプリングする

    Args:
        df: クラスタリング結果のDataFrame
        cluster_column: クラスタIDが格納されている列名
        exclude_cluster_id: 除外するクラスタID
        text_column: テキストデータが格納されている列名
        sample_size: 各クラスタからサンプリングするデータ数
        sample_clusters: サンプリングするクラスタ数（Noneの場合は全クラスタ）

    Returns:
        サンプリングしたテキストデータのリスト
    """
    other_clusters = df[df[cluster_column] != exclude_cluster_id][cluster_column].unique()
    
    if len(other_clusters) == 0:
        return []
    
    if sample_clusters is not None and sample_clusters < len(other_clusters):
        other_clusters = random.sample(list(other_clusters), sample_clusters)
    
    sampled_data = []
    for cluster_id in other_clusters:
        cluster_samples = sample_cluster_data(
            df, cluster_column, cluster_id, text_column, sample_size
        )
        sampled_data.extend(cluster_samples)
    
    return sampled_data


def evaluate_cluster_title(
    prompt: str,
    cluster_title: str,
    cluster_samples: list[str],
    other_clusters_samples: list[str],
    model: str = "gpt-4o"
) -> dict[str, dict]:
    """クラスタのタイトルを評価する

    Args:
        prompt: 評価用プロンプト
        cluster_title: 評価対象のクラスタタイトル
        cluster_samples: 評価対象クラスタのサンプルデータ
        other_clusters_samples: 他のクラスタのサンプルデータ
        model: 使用するLLMモデル名

    Returns:
        評価結果（JSON形式）
    """
    # 入力データの整形
    input_text = f"評価対象クラスタのタイトル: 「{cluster_title}」\n\n"
    
    input_text += "評価対象クラスタに所属するテキストデータのサンプル:\n"
    for sample in cluster_samples:
        input_text += f"- {sample}\n"
    
    input_text += "\n他のクラスタからサンプリングしたテキストデータ:\n"
    for sample in other_clusters_samples:
        input_text += f"- {sample}\n"
    
    # LLMに評価リクエスト
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": input_text},
    ]
    
    try:
        response = request_to_chat_openai(messages=messages, model=model, is_json=True)
        evaluation = json.loads(response)
        return evaluation
    except Exception as e:
        print(f"評価中にエラーが発生しました: {e}")
        return {
            "error": str(e),
            "title_evaluation": {
                "inclusiveness_score": {"score": 0, "reason": "評価エラー"},
                "specificity_score": {"score": 0, "reason": "評価エラー"},
                "concreteness_score": {"score": 0, "reason": "評価エラー"},
                "overall_score": {"score": 0, "reason": "評価エラー"},
            }
        }


def evaluate(
    prompt: str,
    clusters_df: pd.DataFrame,
    cluster_column: str,
    text_column: str,
    sample_size: int,
    other_sample_size: int,
    other_sample_clusters: int | None = None,
    max_workers: int | None = None,  # 並列数を指定（Noneなら自動）
) -> list[dict[str, dict]]:
    cluster_ids = sorted(clusters_df[cluster_column].unique())
    evaluation_results = []

    def eval_one(cluster_id):
        cluster_title = clusters_df[clusters_df[cluster_column] == cluster_id]["cluster-level-2-label"].values[0]
        cluster_samples = sample_cluster_data(clusters_df, cluster_column, cluster_id, text_column, sample_size)
        other_clusters_samples = sample_other_clusters_data(
            clusters_df, cluster_column, cluster_id, text_column, other_sample_size, other_sample_clusters
        )
        return {
            "id": cluster_id,
            "label": cluster_title,
            **evaluate_cluster_title(prompt, cluster_title, cluster_samples, other_clusters_samples)
        }

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(eval_one, cluster_id): cluster_id for cluster_id in cluster_ids}
        for future in tqdm(as_completed(futures), total=len(futures)):
            result = future.result()
            evaluation_results.append(result)

    return evaluation_results

In [53]:
cluster_column = "cluster-level-1-id"
cluster_ids = sorted(clusters_df[cluster_column].unique())


In [54]:
eval_results = evaluate(
    prompt=EVALUATION_PROMPT,
    clusters_df=clusters_df,
    cluster_column="cluster-level-1-id",
    text_column="argument",
    sample_size=30,
    other_sample_size=4,
    other_sample_clusters=10
)

100%|██████████| 15/15 [00:09<00:00,  1.56it/s]


In [55]:
def parse_val(val: dict | str):
    if isinstance(val, dict):
        return val["score"]
    return val

In [56]:
eval_score_df = pd.DataFrame([
        {
            key: parse_val(val_dict)
            for key, val_dict in eval_result.items()} 
            for eval_result in eval_results
            
    ]
)

In [57]:
eval_score_df.sort_values(by="id")

,id,label,inclusiveness_score,specificity_score,concreteness_score
4,1_1,著作権保護の強化と法的整備の必要性に関する懸念と提案,5,4,4
2,1_10,生成AIと著作権に関する倫理的・法的懸念,5,4,4
5,1_11,生成AIと著作権侵害に関する倫理的・法的懸念,5,4,4
1,1_12,生成AIによるクリエイターへの影響と規制の必要性,5,4,4
10,1_13,AI生成物と手描き作品の識別と著作権保護の必要性,5,4,4
12,1_14,AI生成物の透明性と信頼性確保のための表記義務と規制の必要性,5,4,4
9,1_15,AI技術による著作権・肖像権侵害と倫理的懸念,5,4,4
6,1_2,クリエイターの権利保護と文化産業の持続可能性,5,4,4
7,1_3,AIと著作権に関する倫理的・法的課題の検討,5,4,4
3,1_4,AI生成物と著作権に関する倫理的・法的課題,5,4,4


In [58]:
eval_score_df[["inclusiveness_score", "specificity_score", "concreteness_score"]].mean(axis=0)

inclusiveness_score    4.933333
specificity_score      3.933333
concreteness_score     3.866667
dtype: float64

In [59]:
print("平均スコア", eval_score_df[["inclusiveness_score", "specificity_score", "concreteness_score"]].mean().mean())

平均スコア 4.2444444444444445


In [60]:
eval_score_df.sort_values(by="id").to_csv(exp_output_dir / "プロンプト変更後の自動評価結果.csv", index=False)